# Lab 4 – LoRA Fine-Tuning
**Part 4 of the 7 Lab Hands-On SLM Training Series**

In this lab, we fine-tune a Small Language Model (SLM) on your domain data using **LoRA** (Low‑Rank Adaptation) with the `peft` library. This notebook is designed to be stable in Google Colab and to load the processed dataset saved in Lab 3 from Google Drive.

**Outcome**
• Attach LoRA adapters to a base model
• Run a short fine‑tuning loop on your tokenized dataset
• Save the LoRA adapters back to Drive for reuse


## Step 0. Stable installs for Colab

In [1]:
%pip install -q --force-reinstall "numpy==2.0.2" "pandas==2.2.2" "pyarrow==17.0.0"
%pip install -q "datasets>=3.0.0" "transformers>=4.41.0" "peft>=0.11.0" "accelerate>=0.29.0" "sentencepiece>=0.1.99" "tqdm>=4.66.0" bitsandbytes

import importlib, traceback
mods = ["numpy", "pandas", "pyarrow", "datasets", "transformers", "peft", "accelerate", "sentencepiece", "tqdm"]
for m in mods:
    try:
        mod = importlib.import_module(m)
        print(f"{m}: {getattr(mod, '__version__', 'unknown')}")
    except Exception as e:
        print(f"[Import error] {m}: {e}")
print("If any import failed, go to Runtime → Restart runtime, then re-run this cell.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.4 MB/s eta 0:00:00
numpy: 2.0.2
pandas: 2.2.2
pyarrow: 17.0.0
datasets: 4.0.0
transformers: 4.55.2
peft: 0.17.0
accelerate: 1.10.0
sentencepiece: 0.2.1
tqdm: 4.67.1
If any import failed, go to Runtime → Restart runtime, then re-run this cell.


## Step 1. Load the prepared dataset from Google Drive

In [2]:
from datasets import load_from_disk
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/slm-labs/lab3_tokenized"  # Path where Lab 3 saved the tokenized dataset
dataset = load_from_disk(DATA_DIR)
print(dataset)
print("Train rows:", len(dataset["train"]))
print("Columns:", dataset["train"].column_names)

Mounted at /content/drive
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 121736
    })
})
Train rows: 121736
Columns: ['input_ids', 'attention_mask', 'labels']


## Step 2. Load a base model (4‑bit on GPU if available)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Choose a public model that works well with chat templates and LoRA
PREFERRED_MODELS = [
    "HuggingFaceH4/zephyr-7b-beta",
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
]

def load_base_model(name: str):
    use_gpu = torch.cuda.is_available()
    print(f"CUDA available: {use_gpu}")
    quant_cfg = None
    kwargs = {}
    if use_gpu:
        try:
            quant_cfg = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
            )
            kwargs.update(dict(device_map="auto", quantization_config=quant_cfg, torch_dtype=torch.float16))
        except Exception as e:
            print("bitsandbytes not available, falling back to non-quantized load.")
            kwargs.update(dict(torch_dtype=torch.float16 if use_gpu else torch.float32))
    else:
        kwargs.update(dict(torch_dtype=torch.float32))

    tok = AutoTokenizer.from_pretrained(name, use_fast=True)
    mdl = AutoModelForCausalLM.from_pretrained(name, **kwargs)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok, mdl

tokenizer = model = None
last_err = None
for cand in PREFERRED_MODELS:
    try:
        print(f"Attempting model: {cand}")
        tokenizer, model = load_base_model(cand)
        model_name = cand
        print(f"Loaded: {cand}")
        break
    except Exception as e:
        last_err = e
        print(f"Failed to load {cand}: {e}")

if model is None:
    raise RuntimeError(f"Could not load any model. Last error: {last_err}")

Attempting model: HuggingFaceH4/zephyr-7b-beta
CUDA available: True


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Loaded: HuggingFaceH4/zephyr-7b-beta


## Step 3. Attach LoRA adapters with PEFT

In [4]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for k-bit training if quantized
model = prepare_model_for_kbit_training(model)

# Common LoRA target modules for decoder-only models (LLaMA/Mistral/Zephyr families)
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758


## Step 4. Fine‑tune with transformers.Trainer

In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import math

# Use validation split if present; otherwise train only
train_ds = dataset["train"]
eval_ds = dataset.get("validation") or dataset.get("test")

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.0,
    warmup_steps=10,
    max_steps=100,  # keep small for demo; increase for real training
    logging_steps=10,
    save_strategy="no",
    fp16=torch.cuda.is_available(),
    bf16=False,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collator,
)

train_result = trainer.train()
print(train_result)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.679500
20,1.632000
30,1.636800
40,1.630000
50,1.608000
60,1.644200
70,1.613100
80,1.601900
90,1.576600
100,1.605900


TrainOutput(global_step=100, training_loss=1.6227950954437256, metrics={'train_runtime': 3530.4019, 'train_samples_per_second': 0.227, 'train_steps_per_second': 0.028, 'total_flos': 3.51564749340672e+16, 'train_loss': 1.6227950954437256, 'epoch': 0.006571597555365709})


## Step 5. Save LoRA adapters to Google Drive

In [6]:
from pathlib import Path
SAVE_DIR = "/content/drive/MyDrive/slm-labs/lab4_lora_adapters"
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("Saved LoRA adapters to:", SAVE_DIR)

Saved LoRA adapters to: /content/drive/MyDrive/slm-labs/lab4_lora_adapters


## Optional: Quick generation check

In [7]:
from transformers import TextStreamer

prompt = "Summarize the key considerations when drafting a cardiology discharge note."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

with torch.no_grad():
    _ = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        streamer=streamer,
        pad_token_id=tokenizer.eos_token_id,
    )

Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting 

Answer:468. 

Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient

The 

Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient

...

and 

Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient

the 

Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in MistralDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient

moundary


### Wrap‑up
You have:
• Loaded your tokenized dataset from Google Drive
• Attached LoRA adapters with PEFT
• Run a short fine‑tuning loop using `transformers.Trainer`
• Saved adapters back to Drive for reuse in inference or future training

Next up: **Lab 5 – Hyperparameter Tuning and Optimization**.